In [1]:
import src.requestcompletion as rc
from typing import List
from src.requestcompletion.visuals.agent_viewer import AgentViewer

In [2]:
def available_locations() -> List[str]:
    """Returns a list of available locations.
    Args:
    Returns:
        List[str]: A list of available locations.
    """
    return [
        "New York",
        "Los Angeles",
        "Chicago",
        "Delhi",
        "Mumbai",
        "Bangalore",
        "Paris",
        "Denmark",
        "Sweden",
        "Norway",
        "Germany",
    ]

def currency_used(location: str) -> str:
    """Returns the currency used in a location.
    Args:
        location (str): The location to get the currency used for.
    Returns:
        str: The currency used in the location.
    """
    currency_map = {
        "New York": "USD",
        "Los Angeles": "USD",
        "Chicago": "USD",
        "Delhi": "INR",
        "Mumbai": "INR",
        "Bangalore": "INR",
        "Paris": "EUR",
        "Denmark": "EUR",
        "Sweden": "EUR",
        "Norway": "EUR",
        "Germany": "EUR",
    }
    used_currency = currency_map.get(location)
    if used_currency is None:
        raise ValueError(f"Currency not available for location: {location}")
    return used_currency

def average_location_cost(location: str, num_days: int) -> float:
    """Returns the average cost of living in a location for a given number of days.
    Args:
        location (str): The location to get the cost of living for.
        num_days (int): The number of days for the trip.
    Returns:
        float: The average cost of living in the location.
    """
    daily_costs = {
        "New York": 200.0,
        "Los Angeles": 180.0,
        "Chicago": 150.0,
        "Delhi": 50.0,
        "Mumbai": 55.0,
        "Bangalore": 60.0,
        "Paris": 220.0,
        "Denmark": 250.0,
        "Sweden": 240.0,
        "Norway": 230.0,
        "Germany": 210.0,
    }
    daily_cost = daily_costs.get(location)
    if daily_cost is None:
        raise ValueError(f"Cost information not available for location: {location}")
    return daily_cost * num_days

def convert_currency(amount: float, from_currency: str, to_currency: str) -> float:
    """Converts currency using a static exchange rate (for testing purposes).
    Args:
        amount (float): The amount to convert.
        from_currency (str): The currency to convert from.
        to_currency (str): The currency to convert to.
    Returns:
        float: The converted amount.
    Raises:
        ValueError: If the exchange rate is not available.
    """
    exchange_rates = {
        ("USD", "EUR"): 0.85,
        ("EUR", "USD"): 1.1765,
        ("USD", "INR"): 83.0,
        ("INR", "USD"): 0.01205,
        ("EUR", "INR"): 98.0,
        ("INR", "EUR"): 0.0102,
    }

    rate = exchange_rates.get((from_currency, to_currency))
    if rate is None:
        raise ValueError("Exchange rate not available")
    return amount * rate



In [3]:
convert_currency_node = rc.library.from_function(convert_currency)
available_locations_node = rc.library.from_function(available_locations)
currency_used_node = rc.library.from_function(currency_used)
average_location_cost_node = rc.library.from_function(average_location_cost)    

In [4]:
travel_planner_node = rc.library.tool_call_llm(connected_nodes={convert_currency_node, available_locations_node, currency_used_node, average_location_cost_node}, 
                                               pretty_name="Travel Planner Node", 
                                               system_message=rc.llm.SystemMessage("You are a travel planner that will plan a trip. you have access to AvailableLocations, ConvertCurrency, CurrencyUsed and AverageLocationCost tools. Use them when you need to."),
                                               model=rc.llm.OpenAILLM("gpt-4o"))

In [5]:
with rc.Runner() as runner:
    result = await runner.run(travel_planner_node, message_history=rc.llm.MessageHistory([rc.llm.UserMessage("I live in Delhi. I am going to travel to Denmark for 3 days, followed by Germany for 2 days and finally New York for 4 days. Please provide me with a budget summary for the trip in INR.")]))

[+3.473  s] RC.RUNNER   : INFO     - START CREATED Travel Planner Node - (, message_history=user: I live in Delhi. I am going to travel to Denmark for 3 days, followed by Germany for 2 days and finally New York for 4 days. Please provide me with a budget summary for the trip in INR.)
[+5.235  s] RC.RUNNER   : INFO     - Travel Planner Node CREATED currency_used Node - ({'location': 'Denmark'}, )
[+5.238  s] RC.RUNNER   : INFO     - currency_used Node DONE EUR
[+6.372  s] RC.RUNNER   : INFO     - Travel Planner Node CREATED currency_used Node - ({'location': 'Germany'}, )
[+6.373  s] RC.RUNNER   : INFO     - currency_used Node DONE EUR
[+7.525  s] RC.RUNNER   : INFO     - Travel Planner Node CREATED currency_used Node - ({'location': 'New York'}, )
[+7.527  s] RC.RUNNER   : INFO     - currency_used Node DONE USD
[+8.426  s] RC.RUNNER   : INFO     - Travel Planner Node CREATED average_location_cost Node - ({'location': 'Denmark', 'num_days': 3}, )
[+8.427  s] RC.RUNNER   : INFO     - ave

In [6]:
viewer = AgentViewer(
            result.all_stamps,
            result.request_heap,
            result.node_heap
        )
viewer.display_graph()